In [1]:
import os
from typing import TypedDict, Annotated, List, Literal
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_tavily import TavilySearch
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, SystemMessage
from typing import Annotated, Sequence, TypedDict, List
from langchain_core.prompts import PromptTemplate
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, START, END



In [2]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    specialist_messages: Annotated[Sequence[BaseMessage], add_messages]
    patho_messages: Annotated[Sequence[BaseMessage], add_messages]
    radio_messages: Annotated[Sequence[BaseMessage], add_messages]
    patho_QnA : list[str]
    radio_QnA : list[str]


patient_info = ""
final_report = ""


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv
load_dotenv()

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=os.getenv("GEMINI_API_KEY"),
    temperature=0.0,
    
)

In [4]:
# lst = ['hi', 'hello', HumanMessage(content="hi"), AIMessage(content="hello"),HumanMessage(content='tell which message was by Human and which was by AI in whole conversation")]')]
# llm.invoke(lst)

In [5]:
# from langchain_groq import ChatGroq
# llm = ChatGroq(model="llama-3.3-70b-versatile", api_key="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")

In [6]:
tavily_search = TavilySearch(
    max_results=5,
    search_depth="basic",
)

In [7]:
# Corrected version for cell 7
@tool
def ask_user(question: str) -> str:
    """Ask the user a question and get their response."""
    try:
        print(f"\n🏥 DOCTOR ASKS: {question}")
        print("=" * 50)
        
        # For Jupyter, use a more robust input method
        import sys
        if 'ipykernel' in sys.modules:
            # In Jupyter
            response = input("YOUR ANSWER: ")
        else:
            # In regular Python
            response = input("YOUR ANSWER: ")
            
        if not response.strip():
            return "Patient did not provide an answer."  
        return response.strip()
    except (EOFError, KeyboardInterrupt):
        return "Patient ended the consultation."
    except Exception as e:
        return f"Unable to get patient response: {str(e)}"

In [8]:
@tool
def search_internet(query: str) -> str:
    """Search the internet for the given query and return the results.
    Args:
        query (str): The search query.
    Returns:
        str: The search results.
    """
    result = tavily_search.invoke({"query": query})
    return str(result)

In [9]:
@tool
def Patient_data_report(data: str)->str:
    """Generate a report from the patient data."""
    # Here you would implement the logic to create a report from the patient data
    global patient_info
    patient_info = data
    return "Patient Data compiled, Recommend a Specialist"

In [10]:
gp_llm = llm.bind_tools([ask_user, Patient_data_report])
ophthallm = llm.bind_tools([ask_user, search_internet])
pathllm = llm.bind_tools([ask_user, search_internet])   
radllm = llm.bind_tools([ask_user, search_internet])    

In [ ]:
def general_physician(state: AgentState) -> AgentState:
    # Implement the logic for the general physician agent
    global patient_info
    SystemPrompt = SystemMessage(content=f"""You are a general physician.
    You MUST ALWAYS use the ask_user tool to obtain ANY patient information (never ask questions directly in plain text).
Process:
1. If you still need ANY detail (symptoms, duration, location, severity, age, name, relevant history, child context) you MUST call ask_user (one question per call).
2. Only AFTER you have enough information, call Patient_data_report with a concise structured summary (demographics + key symptoms + relevant negatives).
3. Just AFTER Patient_data_report has been called: Patient_data_report status: {bool(patient_info)} is True, output EXACTLY specialist name, return ONLY the name, nothing else.
Rules:
- Do NOT guess age or other details—always ask via ask_user.
- Do NOT output a specialist name until Patient_data_report tool has been called by you.
- Never ask multiple questions in one tool call; break them up if needed, one tool call a time.
If it's a child: ensure you collected child's age, weight, height explicitly first.

""")

    response = gp_llm.invoke([SystemPrompt] + state['messages'])
    return {'messages': [response]}

In [12]:
def router_gp(state: AgentState) -> AgentState:
    last_message = state['messages'][-1]
    content = last_message.content.lower()
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "GP_Tooler"
    else:
        return "ophthalmology"

In [ ]:
def Ophthalmologist(state: AgentState) -> AgentState:
    # Implement the logic for the ophthalmologist agent
    global patient_info
    SystemPrompt = SystemMessage(content=f"""You are a High Quality Ophthalmologist.

Your patient's initial data is: {patient_info}.

Current status:
1. Status of Radiologist QnA: {", ".join(state['radio_QnA']) if state['radio_QnA'] else "None"}.
2. Status of Pathologist QnA: {", ".join(state['patho_QnA']) if state['patho_QnA'] else "None"}.
3. Current report status: {", ".join(state['current_report']) if state['current_report'] else "None"}.

You have access to three tools:
1. **ask_user** – Use this tool to ask the patient any questions you need answered. 
2. **search_internet** – Use this tool to look up any medical information you need. 
3. **add_report** – Use this tool to add relevant findings to the report. You can call it multiple times. 'current_report' will include Pathologist and Radiologist findings after you request their help.

Your tasks:
1. **Ask Questions**: If more patient information is needed, you MUST use the 'ask_user' tool. Ask one question at a time.
2. **Use Helpers**: If you need a Pathologist or Radiologist, output plain text like: 
   - "I need a blood report from Pathologist, (your question)"
   - "I need imaging studies from Radiologist, (your question)"
3. **Final Report**: Only after completing the diagnosis, output plain text starting with the exact phrase: 
   - "Final Report: take finalised report from current_report"
4. The final report must include: Symptoms, Diagnosis, Treatment Plan, Follow-up Instructions. If anything is missing from 'current report', call 'add_report' first, then produce the Final Report.

Rules:
- Always use the 'ask_user' tool for questions to the patient.
- The conversation continues until a Final Report is produced.
- Responses from Pathologist or Radiologist will automatically be added to your context.
- Never ask multiple questions in one tool call.
- If you return plain text that does not mention 'pathologist', 'radiologist', or 'Final Report:', it will be ignored.

""")

    
    response = ophthallm.invoke([SystemPrompt]+state['specialist_messages']) 
    return {'specialist_messages' : [response]}

In [ ]:
def router_opthal(state: AgentState) -> AgentState:
    # Route the request to the appropriate ophthalmologist agent
    global final_report
    last_message = state['specialist_messages'][-1]
    content = last_message.content.lower()

    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        for tool_call in last_message.tool_calls:
            if tool_call.name == 'add_report':
                state['current_report'].append(last_message.content)
        return "Ophthal_Tooler"
    
    elif "pathologist" in content:
        state["patho_QnA"].append("Question from Ophthalmologist to Pathologist: ")
        state["patho_QnA"].append(last_message.content)
        return "Pathologist"
    elif "radiologist" in content:
        state['radio_QnA'].append("Question from Ophthalmologist to Radiologist: ")
        state['radio_QnA'].append(last_message.content)
        return "Radiologist"
    elif "final report:" in content:
        global final_report 
        final_report = last_message.content
        return "end"
    else:
        return "Ophthalmologist"

<h2 style="color:LightBlue;">Helper Agents</h2>
<p><span style="font-weight:bold; color:green;">Pathologist</span> 🧪 & 
<span style="font-weight:bold; color:orange;">Radiologist</span> 🩻</p>


In [15]:
def Pathologist(state: AgentState) -> AgentState:
    # Implement the logic for the pathologist agent
    global patient_info
    SystemPrompt = SystemMessage(content=f"""You are a High Quality Pathologist.
    Currently you have patient report and current status present in {patient_info}.
    1) Any information required, You have to use 'ask_user' tool to ask user about any details which might help you, use 'ask_user' tool as many times as required.
    2) You have 'search_internet' tool to search internet for any query at any point of time in analysis.
    You are helper Pathologist for specialists. You ask user to provide blood report and any information that special might have suspected and asked you to ask user for.
    If you had any conversation with Specialist before, their status will be his question and your reply as a string, else it will be 'None':
    Your total Conversation with specialist is presented as 1 string: {", ".join(state['patho_QnA']) if state['patho_QnA'] else "None"}, based on this, if ending of string is a question by specialist, frame your response(you can also use tools if needed).
    The tools you use will always return back to you, so do not show any hurry and use all tools at your disposal any number of times.
    After you have framed satisfactory answer to provide back to specialist, return output as PLAIN TEXT not tool call in following: 'This is the final report to specialist from Pathology labs: (You final summary)'. Follow EXACT format.
    If you do not call for a tool and your plain output do not contain 'This is the final report to specialist from Pathology labs:', it will be ignored and you will be asked to respond again. Do not repeat this mistake otherwise it will be inifnite loop.
      """)

    response = pathllm.invoke([SystemPrompt]+ state["patho_messages"])
    return {'patho_messages' : [response]}

In [16]:
def router_patho(state: AgentState) -> AgentState:
    # Route the request to the appropriate pathologist agent
    last_message = state['patho_messages'][-1]
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "Patho_Tooler"
    elif "final report" in last_message.content.lower() and "specialist" in last_message.content.lower():
        state["patho_QnA"].append("Pathologist Answer report to specialist:")
        state["patho_QnA"].append(last_message.content)
        return "Ophthalmologist"
    else:
        return "Pathologist"

In [17]:
def Radiologist(state: AgentState) -> AgentState:
    # Implement the logic for the radiologist agent
    global patient_info
    SystemPrompt = SystemMessage(content=f"""You are a High Quality Radiologist.
    Currently you have patient report and current status present in {patient_info}.
    1)Any information required, You have to use 'ask_user' tool to ask user about any details which might help you, use 'ask_user' tool as many times as required.
    2) You have 'search_internet' tool to search internet for any query at any point of time in analysis.
    You are helper Radiologist for specialists. You ask user to provide XRAY reports and any information that special might have suspected and asked you to ask user for.
    If you had any conversation with Specialist before, their status will be his question and your reply as a string, else it will be 'None':
    Your total Conversation with specialist is presented as 1 string: {", ".join(state['radio_QnA']) if state['radio_QnA'] else "None"}, based on this, if ending of string is a question by specialist, frame your response(you can also use tools if needed).
    The tools you use will always return back to you, so do not show any hurry and use all tools at your disposal any number of times.
    After you have framed satisfactory answer to provide back to specialist, return output as PLAIN TEXT not tool call in following: 'This is the final report to specialist from Radiology labs: (You final summary)'. Follow EXACT format.
    If you do not call for a tool and your plain output do not contain 'This is the final report to specialist from Radiology labs:', it will be ignored and you will be asked to respond again. Do not repeat this mistake otherwise it will be inifnite loop.
      """)

    response = pathllm.invoke([SystemPrompt]+ state["radio_QnA"])
    return {'radio_messages' : [response]}

In [18]:
def router_radio(state: AgentState) -> AgentState:
    # Route the request to the appropriate radiologist agent
    last_message = state['radio_messages'][-1]
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "Radio_Tooler"
    elif "final report" in last_message.content.lower() and "specialist" in last_message.content.lower():
        state["radio_QnA"].append("Radiologist Answer report to specialist:")
        state["radio_QnA"].append(last_message.content)
        return "Ophthalmologist"
    else:
        return "Radiologist"

In [19]:
gp_tools = [ask_user, Patient_data_report]
opthal_tools = [ask_user, search_internet]
patho_tools = [ask_user, search_internet]
radio_tools = [ask_user, search_internet]


In [ ]:
opthal_tool_node = ToolNode(opthal_tools)

# The corrected "adapter" node for the graph
def opthal_tool_invoker(state: AgentState) -> dict:
    """
    Takes tool calls from 'specialist_messages', runs them,
    and returns the output to be added back to 'specialist_messages'.
    """
    # Create the dictionary input the ToolNode expects
    tool_input = {'messages': [state['specialist_m essages'][-1]]}
    
    # Run the standard ToolNode. It returns a dictionary like {'messages': [ToolMessage(...)]}
    tool_output_dict = opthal_tool_node.invoke(tool_input)
    
    # *** THIS IS THE NEW LINE ***
    # We must extract the list of messages from that dictionary
    tool_output_messages = tool_output_dict['messages']
    
    # Return the list under the correct key for our specialist agent
    return {'specialist_messages': tool_output_messages}

In [21]:
graph = StateGraph(AgentState)
graph.add_node("GP", general_physician)
graph.add_node("Ophthalmologist", Ophthalmologist)

GP_Tooler = ToolNode(tools=gp_tools)
graph.add_node("GP_Tooler", GP_Tooler)
graph.add_node("Ophthal_Tooler", opthal_tool_invoker)

graph.add_node("Pathologist", Pathologist)
graph.add_node("Radiologist", Radiologist)
Patho_Tooler = ToolNode(tools=patho_tools)
Radio_Tooler = ToolNode(tools=radio_tools)
graph.add_node("Patho_Tooler", Patho_Tooler)
graph.add_node("Radio_Tooler", Radio_Tooler)
graph.add_edge("Patho_Tooler", "Pathologist")
graph.add_edge("Radio_Tooler", "Radiologist")




graph.add_edge(START, "GP")
graph.add_edge("GP_Tooler", "GP")
graph.add_edge("Ophthal_Tooler", "Ophthalmologist")


graph.add_conditional_edges(
    "GP",
    router_gp,
    {
        "GP_Tooler": "GP_Tooler",
        "ophthalmology": "Ophthalmologist",
    }
)
graph.add_conditional_edges(
    "Ophthalmologist",
    router_opthal,
    {
        "Ophthal_Tooler": "Ophthal_Tooler",
        "Pathologist": "Pathologist",
        "Radiologist": "Radiologist",
        "Ophthalmologist": "Ophthalmologist",
        "end": END
    }
)

graph.add_conditional_edges(
    "Pathologist",
    router_patho,
    {
        "Patho_Tooler": "Patho_Tooler",
        'Ophthalmologist': 'Ophthalmologist',
        "Pathologist": "Pathologist"
    }
)
graph.add_conditional_edges(
    "Radiologist",
    router_radio,
    {
        "Radio_Tooler": "Radio_Tooler",
        'Ophthalmologist': 'Ophthalmologist',
        "Radiologist": "Radiologist"
    }
)

In [22]:
# Compile the graph
app = graph.compile()

In [23]:
# Suppose `app` is your compiled LangGraph
inputs = {"messages": [HumanMessage(content="Hello, I am your patient")],
    "specialist_messages": [HumanMessage(content="Hello, I am your patient")],
    "patho_messages": [HumanMessage(content="Generate some test based on status of Pathology status")],
    "radio_messages": [HumanMessage(content="Generate some report based on status of Radiology status")],
    "patho_QnA": [],
    "radio_QnA": [],}

for event in app.stream(inputs,config = {"recursion_limit": 50}):
    # event is a dict keyed by node name
    for node, payload in event.items():
        print(f"\n=== Node executed: {node} ===")
        print("Output:", payload)



=== Node executed: GP ===
Output: {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'ask_user', 'arguments': '{"question": "What is your main symptom?"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--29d21758-41a9-4922-b3fc-30469ecb2937-0', tool_calls=[{'name': 'ask_user', 'args': {'question': 'What is your main symptom?'}, 'id': '11ca5bc9-7549-4bd8-966f-ec971c369c70', 'type': 'tool_call'}], usage_metadata={'input_tokens': 325, 'output_tokens': 60, 'total_tokens': 385, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 40}})]}

🏥 DOCTOR ASKS: What is your main symptom?

=== Node executed: GP_Tooler ===
Output: {'messages': [ToolMessage(content='eye redness', name='ask_user', id='fc1083a3-81ae-4d34-81d6-46a3af69495b', tool_call_id='11ca5bc9-7549-4bd8-966f-ec971c369c70')]}

=== Node executed: GP ===
O

In [24]:
print(patient_info)

Patient is a 58-year-old presenting with eye redness for 2 days, with a severity of 3/10.


In [25]:
print(final_report)

Final Report: The patient presents with a 2-day history of gradual onset, mild (3/10) redness in the left eye, without pain, itching, discharge, or vision changes. The patient reports rubbing the eye and recent exposure to dust while cleaning. These symptoms are consistent with irritant or allergic conjunctivitis, likely triggered by dust exposure and exacerbated by eye rubbing. No further investigations are immediately required. I recommend artificial tears for lubrication and to help flush out any remaining irritants, and advise the patient to avoid rubbing the eye. If symptoms worsen or new symptoms develop, the patient should return for re-evaluation.


In [26]:
print(app.get_graph().draw_mermaid())


---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	GP(GP)
	Ophthalmologist(Ophthalmologist)
	GP_Tooler(GP_Tooler)
	Ophthal_Tooler(Ophthal_Tooler)
	Pathologist(Pathologist)
	Radiologist(Radiologist)
	Patho_Tooler(Patho_Tooler)
	Radio_Tooler(Radio_Tooler)
	__end__([<p>__end__</p>]):::last
	GP -.-> GP_Tooler;
	GP -. &nbsp;ophthalmology&nbsp; .-> Ophthalmologist;
	GP_Tooler --> GP;
	Ophthal_Tooler --> Ophthalmologist;
	Ophthalmologist -.-> Ophthal_Tooler;
	Ophthalmologist -.-> Pathologist;
	Ophthalmologist -.-> Radiologist;
	Ophthalmologist -. &nbsp;end&nbsp; .-> __end__;
	Patho_Tooler --> Pathologist;
	Pathologist -.-> Ophthalmologist;
	Pathologist -.-> Patho_Tooler;
	Radio_Tooler --> Radiologist;
	Radiologist -.-> Ophthalmologist;
	Radiologist -.-> Radio_Tooler;
	__start__ --> GP;
	Ophthalmologist -.-> Ophthalmologist;
	Pathologist -.-> Pathologist;
	Radiologist -.-> Radiologist;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first 